In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import kendalltau
import scipy as sp
import sklearn as sk
import json
import copy 
pd.set_option('display.max_rows', 30)

In [ ]:
jup_conversion = {'Lowest': 1,
                  'Low': 2,
                 'Medium': 3,
                 'High': 4,
                 'Highest': 5}

In [ ]:
def load_jupiter_file(file_path, variable):
    jupiter = pd.read_csv(file_path)
    full_df_jupiter = pd.DataFrame()
    for scenario in ['SSP1-2.6 (1.8C)', 'SSP2-4.5 (2.7C)','SSP5-8.5 (4.4C)']:
        scenario_subset = jupiter[jupiter['scenario']==scenario]
        # use the pivot_table utility in pandas to convert the locations into the index values
        # and the time and scenario as the columns
        scenario_subset = scenario_subset.pivot_table(index='locationId', 
                                                      columns='year', 
                                                      values=variable, 
                                                      aggfunc='first', 
                                                      dropna=False)
        scenario_subset.columns = [f'{scenario} - {year}' for year in np.arange(2020, 2101, 5)]
        full_df_jupiter = pd.concat([full_df_jupiter, scenario_subset], axis=1)
    full_df_jupiter['Historical baseline - 1995'] = jupiter[jupiter['scenario']=='Historical Baseline'][variable].values
    full_df_jupiter.index = full_df_jupiter.index-1
    return full_df_jupiter

In [ ]:
def load_jupiter_ca(risks):
    _data = load_jupiter_file('s3://carbonplan-climate-impacts/climate-risk-comparison/companies/data/CA-Fire-for-CarbonPlan-Input_CA-Fire-for-CarbonPlan_Enhanced_20240502.csv',
                            'FR_annualFireProbability_tier')
    for col in _data.columns:
        _data[col] = _data[col].map(jup_conversion)
    risks['ca']['jupiter'] = locations['ca'].join(_data)[['geometry', 
                                                           'Historical baseline - 1995', 
                                                           'SSP5-8.5 (4.4C) - 2100']]
    return risks

In [ ]:
def load_jupiter_ny(risks, metric='score'):
    risks['nystate']['jupiter'] = {}
    risks['nyc']['jupiter'] = {}
    variable_shortname_conversions = {'PF': 'Pluvial',
                                 'FF': 'Fluvial',
                                 'CF': 'Coastal'}
    for variable in ['PF', 'CF', 'FF']:
        # use the 100-year return period for flooding as it is 
        _data = load_jupiter_file('s3://carbonplan-climate-impacts/climate-risk-comparison/companies/data/NY-Flood-for-CarbonPlan-Input_NY-Flood-for-CarbonPlan_Enhanced_20240502.csv',
                             f'{variable}_depth100yr_tier') 
        for col in _data.columns:
            _data[col] = _data[col].map(jup_conversion) 
        _data.replace(-9999, np.nan, inplace=True)
        locations['ny'].index = np.arange(0,214)
        locations['ny'] = locations['ny']
        # split the nyc and nystate locations
        risks['nystate']['jupiter'][variable_name_conversion_dict[variable]] = locations['ny'].join(_data).iloc[0:124][['geometry', 
                                                                                                                        'Historical baseline - 1995', 
                                                                                                                        'SSP5-8.5 (4.4C) - 2100']]
        risks['nyc']['jupiter'][variable_name_conversion_dict[variable]] = locations['ny'].join(_data).iloc[124:][['geometry', 
                                                                                                                   'Historical baseline - 1995', 
                                                                                                                   'SSP5-8.5 (4.4C) - 2100']]
    return risks

In [ ]:
def load_xdi(region, risks, metric='score'):
    # region is either 'NY' or 'california'
    if region=='NY':
        risks['nyc']['xdi'] = {}
        risks['nystate']['xdi'] = {}
    elif region=='california':
        risks['ca']['xdi'] = {}
    df = pd.read_excel(f's3://carbonplan-climate-impacts/climate-risk-comparison/companies/data/{region}input_merged.xlsx', 
                       sheet_name='hazABC.csv')
    variables_dict = {'california': ['Forest Fire'],
                     'NY': ['Riverine Flooding', 'Coastal Inundation', 'Surface Water Flooding']}
    
    # According to their documentation, XDI has a hazard rating system with three tiers. We assign them
    # to the values 1,3,5 arbitrarily to match the Jupiter five tier scale.
    category_conversion_dict = {'A': 1, # 'Low'
                       'B': 3, # 'Medium'
                       'C': 5} # 'High'
    
    for variable in variables_dict[region]:
        _data = df.pivot_table(index='ID', columns='Year', values=variable, aggfunc='first')
        _data.columns = [f'RCP85-{year}' for year in _data.columns]
        for col in _data.columns:
            _data[col] = _data[col].map(category_conversion_dict)
        if region=='NY':
            _data.index = _data.index.astype('str')
            risks['nystate']['xdi'][variable] = locations['ny'].join(_data).iloc[0:124][['geometry', 'RCP85-1995', 'RCP85-2100']]
            risks['nyc']['xdi'][variable] = locations['ny'].join(_data).iloc[124:][['geometry', 'RCP85-1995', 'RCP85-2100']]
        elif region=='california':
            _data.index -= 1 # index is offset by 1 in the california file
            _data.index = _data.index.astype('str')
            # use 1995 as the year representative of historical
            risks['ca']['xdi'] = locations['ca'].join(_data)[['geometry', 'RCP85-1995', 'RCP85-2100']]
    return risks

In [ ]:
def calculate_kendalltau(xdi, jupiter, variant):    
    tau, p_value = kendalltau(xdi, jupiter, variant=variant)#, nan_policy='omit')
    dist = []

    for i in range(1000):
        indices = sk.utils.resample(np.arange(xdi.shape[0]), replace=True, random_state=i)
        dist.append(sp.stats.kendalltau(xdi.values[indices], jupiter.values[indices], variant=variant)[0])
    dist = np.asarray(dist)
    # Calculate an error bar with one standard deviation
    tau_error = ((np.median(dist) - np.percentile(dist,[16,84])[0]) + (np.percentile(dist,[16,84])[1] - np.median(dist))) / 2
    
    return tau, p_value, tau_error

In [ ]:
def consolidate_risks(risks):
    consolidated = {}
    jup = risks['ca']['jupiter'].rename({'Historical baseline - 1995': 'Jupiter - Fire - California - Historical',
                               'SSP5-8.5 (4.4C) - 2100': 'Jupiter - Fire - California - 2100'}, axis=1)
    xdi = risks['ca']['xdi'].rename({'RCP85-1995': 'XDI - Fire - California - Historical',
                                         'RCP85-2100': 'XDI - Fire - California - 2100'}, axis=1)
    xdi.index = xdi.index.astype('int')
    consolidated['ca'] = pd.concat([jup, xdi[['XDI - Fire - California - Historical',
                               'XDI - Fire - California - 2100']]], axis=1)

    for region in ['nyc', 'nystate']:
        first_flag = True
        for variable in ['Surface Water Flooding', 'Riverine Flooding', 'Coastal Inundation']:
            jup = risks[region]['jupiter'][variable].rename({'Historical baseline - 1995': f'Jupiter - {variable} - {region} - Historical',
                                           'SSP5-8.5 (4.4C) - 2100': f'Jupiter - {variable} - {region} - 2100'}, axis=1)
            xdi = risks[region]['xdi'][variable].rename({'RCP85-1995': f'XDI - {variable} - {region} - Historical',
                                                 'RCP85-2100': f'XDI - {variable} - {region} - 2100'}, axis=1)
            xdi.index = xdi.index.astype('int')
            if first_flag:
                consolidated[region] = pd.concat([jup, xdi[[f'XDI - {variable} - {region} - Historical',
                                       f'XDI - {variable} - {region} - 2100']]], axis=1)
                first_flag = False
            else:
                consolidated[region] = pd.concat([consolidated[region],
                                                  jup[[f'Jupiter - {variable} - {region} - Historical',
                                       f'Jupiter - {variable} - {region} - 2100']], 
                                        xdi[[f'XDI - {variable} - {region} - Historical',
                                       f'XDI - {variable} - {region} - 2100']]], axis=1)
    return consolidated

# Load shapefiles

In [ ]:
states = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/cb_2018_us_state_5m/cb_2018_us_state_5m.shp')

In [ ]:
locations = {}
for state in ['ca', 'ny']:
    locations[state] = gpd.read_file(f's3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/requested_locations_{state}.json')
    locations[state]['ID'] = locations[state].index

# Load Jupiter data

In [ ]:
risks = {'ca': {},
        'nystate': {},
        'nyc': {}}
for region in ['ca', 'ny']:
    locations[region].index = locations[region].index.astype('int')
risks = load_jupiter_ca(risks)
variable_name_conversion_dict = {'PF': 'Surface Water Flooding',
                          'FF':'Riverine Flooding',
                          'CF': 'Coastal Inundation'}
risks = load_jupiter_ny(risks)

# Load XDI data

In [ ]:
for region in ['ca', 'ny']:
    locations[region].index = locations[region].index.astype('str')

risks = load_xdi('california', risks)
risks = load_xdi('NY', risks)

# Write out data to json for web article

In [ ]:
data_for_article = copy.deepcopy(risks)
for company in ['xdi', 'jupiter']:
    data_for_article['ca'][company]['lat'] = data_for_article['ca'][company].geometry.y
    data_for_article['ca'][company]['lon'] = data_for_article['ca'][company].geometry.x
    data_for_article['ca'][company] = data_for_article['ca'][company].drop('geometry', axis=1).to_dict()
for region in ['nyc', 'nystate']:
    for company in ['xdi', 'jupiter']:
        for variable in ['Riverine Flooding', 'Coastal Inundation', 'Surface Water Flooding']:
            data_for_article[region][company][variable]['lat'] = data_for_article[region][company][variable].geometry.y
            data_for_article[region][company][variable]['lon'] = data_for_article[region][company][variable].geometry.x
            data_for_article[region][company][variable] = data_for_article[region][company][variable].drop('geometry', axis=1).to_dict()
with open(f"article_data_revised.json", "w") as outfile:
    json_object = json.dumps(data_for_article, indent=4)
    outfile.write(json_object)

In [ ]:
consolidated_risks = consolidate_risks(risks).copy()

# Calculate percentage above-lowest for each case study

## Fire in California

In [ ]:
time_period_naming_dict = {'xdi': {'historical': 'RCP85-1995',
                                'future': 'RCP85-2100'},
                      'jupiter': {'historical': 'Historical baseline - 1995',
                                'future':  'SSP5-8.5 (4.4C) - 2100'}}

In [ ]:
region = 'California'
variable = 'Fire'
for period in ['Historical', '2100']:
    # remove any locations where there are missing data (NaNs) according to either company
    nans_removed = consolidated_risks['ca'][[f'{company} - {variable} - {region} - {period}' 
                                                        for company in ['Jupiter', 'XDI']]].dropna()
    jupiter_percentage = (nans_removed[f'Jupiter - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'Jupiter - {variable} - {region} - {period}'])
    xdi_percentage = (nans_removed[f'XDI - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'XDI - {variable} - {region} - {period}'])
    
    print(f"percentage above lowest for {variable} in {region} {period} jupiter is :{jupiter_percentage:.2g}, xdi is :{xdi_percentage:.2g}, ")

## Calculating the fraction of locations with increasing fire risk, excluding any which are already the highest risk level in the historical, according to either company.

In [ ]:
not_saturated_jupiter = consolidated_risks['ca']['Jupiter - Fire - California - Historical']!=5
not_saturated_xdi = consolidated_risks['ca']['XDI - Fire - California - Historical']!=5

In [ ]:
not_saturated = consolidated_risks['ca'][not_saturated_jupiter & not_saturated_xdi]

In [ ]:
for company in ['Jupiter', 'XDI']:
    changed = (not_saturated[f'{company} - Fire - California - 2100']>\
               not_saturated[f'{company} - Fire - California - Historical']).sum()/len(not_saturated)
    print(f'Fraction of locations increasing according to {company} is {changed}')

## Coastal inundation in NYC

In [ ]:
region = 'nyc'
variable = 'Coastal Inundation'
for period in ['Historical', '2100']:
            # remove any locations where there are missing data (NaNs) according to either company
            nans_removed = consolidated_risks[region][[f'{company} - {variable} - {region} - {period}' 
                                                                for company in ['Jupiter', 'XDI']]].dropna()
            jupiter_percentage = (nans_removed[f'Jupiter - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'Jupiter - {variable} - {region} - {period}'])
            xdi_percentage = (nans_removed[f'XDI - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'XDI - {variable} - {region} - {period}'])
            
            print(f"percentage above lowest for {variable} in {region} {period} jupiter is :{jupiter_percentage:.2g}, xdi is :{xdi_percentage:.2g}, ")

## Flooding in NY state

In [ ]:
for region in ['nystate', 'nyc']:
    for variable in ['Riverine Flooding', 'Surface Water Flooding']:
        for period in ['Historical', '2100']:
            # remove any locations where there are missing data (NaNs) according to either company
            nans_removed = consolidated_risks[region][[f'{company} - {variable} - {region} - {period}' 
                                                                for company in ['Jupiter', 'XDI']]].dropna()
            jupiter_percentage = (nans_removed[f'Jupiter - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'Jupiter - {variable} - {region} - {period}'])
            xdi_percentage = (nans_removed[f'XDI - {variable} - {region} - {period}']>1).sum()/len(nans_removed[f'XDI - {variable} - {region} - {period}'])
            
            print(f"percentage above lowest for {variable} in {region} {period} jupiter is :{jupiter_percentage:.2g}, xdi is :{xdi_percentage:.2g}, ")

# Calculate fraction of locations that increased in risk in the future

In [ ]:
variable='Riverine Flooding'
period = 'Historical'
region = 'nystate'
for region in ['nystate', 'nyc']:
    for variable in ['Riverine Flooding', 'Surface Water Flooding', 'Coastal Inundation']:
        nans_removed = consolidated_risks[region][[f'{company} - {variable} - {region} - Historical' 
                                                            for company in ['Jupiter', 'XDI']]+[f'{company} - {variable} - {region} - 2100' 
                                                            for company in ['Jupiter', 'XDI']]
                                                ].dropna()
        jupiter_changed = nans_removed[f'Jupiter - {variable} - {region} - 2100']>nans_removed[f'Jupiter - {variable} - {region} - Historical']
        jupiter_percentage = jupiter_changed.sum()/len(nans_removed[f'Jupiter - {variable} - {region} - Historical'])
        xdi_changed = nans_removed[f'XDI - {variable} - {region} - 2100']>nans_removed[f'XDI - {variable} - {region} - Historical']
        xdi_percentage = xdi_changed.sum()/len(nans_removed[f'XDI - {variable} - {region} - Historical'])
        
        
        print(f"for {variable} in {region} according to jupiter is :{jupiter_percentage:.2g}, and xdi is :{xdi_percentage:.2g}, ")

# Calculate Kendall's Tau test statistic for each case study

## Fire in California

In [ ]:
region = 'California'
variable = 'Fire'
for period in ['Historical', '2100']:
    nans_removed = consolidated_risks['ca'][[f'{company} - {variable} - {region} - {period}' 
                                            for company in ['Jupiter', 'XDI']]].dropna()
    tau, p_value, tau_error = calculate_kendalltau(nans_removed[f'Jupiter - {variable} - {region} - {period}'],
                     nans_removed[f'XDI - {variable} - {region} - {period}'], variant='b')
    print(f"Tau for {region} {variable} for {period} is {tau:.2g} +- {tau_error:.2g} with p value {p_value:.2g}")

In [ ]:
region = 'California'
variable = 'Fire'
for period in ['Historical', '2100']:
    nans_removed = consolidated_risks['ca'][[f'{company} - {variable} - {region} - {period}' 
                                            for company in ['Jupiter', 'XDI']]].dropna()
    tau, p_value, tau_error = calculate_kendalltau(nans_removed[f'Jupiter - {variable} - {region} - {period}'],
                     nans_removed[f'XDI - {variable} - {region} - {period}'], variant='c')
    print(f"Tau for {region} {variable} for {period} is {tau:.2g} +- {tau_error:.2g} with p value {p_value:.2g}")

## Coastal inundation in NYC

In [ ]:
region = 'nyc'
variable = 'Coastal Inundation'
for period in ['Historical', '2100']:
    nans_removed = consolidated_risks[region][[f'{company} - {variable} - {region} - {period}' 
                                            for company in ['Jupiter', 'XDI']]].dropna()
    tau, p_value, tau_error = calculate_kendalltau(nans_removed[f'Jupiter - {variable} - {region} - {period}'],
                     nans_removed[f'XDI - {variable} - {region} - {period}'], variant='b')
    print(f"Tau for {region} {variable} for {period} is {tau:.2g} +- {tau_error:.2g} with p value {p_value:.2g}")

## Flooding in New York state

In [ ]:
region = 'nystate'
for variable in ['Surface Water Flooding', 'Riverine Flooding']:
    for period in ['Historical', '2100']:
        nans_removed = consolidated_risks[region][[f'{company} - {variable} - {region} - {period}' 
                                                for company in ['Jupiter', 'XDI']]].dropna()
        tau, p_value, tau_error = calculate_kendalltau(nans_removed[f'Jupiter - {variable} - {region} - {period}'],
                         nans_removed[f'XDI - {variable} - {region} - {period}'], variant='b')
        print(f"Tau for {region} {variable} for {period} is {tau:.2g} +- {tau_error:.2g} with p value {p_value:.2g}")